In [2]:
from __future__ import annotations
import polars as pl
import inspect
import numpy as np
from typing import Tuple

In [2]:
import dsds.metrics as me
import dsds.prescreen as ps

In [ ]:
pred = np.random.random(size=1_000_000) # .reshape(-1, 1) # Predictions
actual = np.round(np.random.random(size=1_000_000)).astype(np.int8)

In [ ]:
actual

In [3]:
from sklearn.datasets import make_classification
orig_x, orig_y = make_classification(n_samples = 100_000, n_features = 50, n_informative = 20, n_redundant = 30)
df_pl = pl.from_numpy(orig_x).insert_at_idx(0, pl.Series("target", orig_y))

In [ ]:

me.logloss(actual, pred, check_binary=False)